In [4]:
import pandas as pd
import numpy as np
from shapely.wkt import loads
from scipy.spatial import KDTree

In [5]:
stations = pd.read_csv('Data/NYC Data/MTA_Subway_Stations_20250124.csv')  # NYC subway stations
pois = pd.read_csv("Data/NYC Data/Points_of_Interest_20250124.csv")          # Points of interest

In [6]:
stations.head()

,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,CBD,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,False,N W,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0,0,0,NaN,POINT (-73.912034 40.775036)
1,R03,2,2,BMT,Astoria,Astoria Blvd,Q,False,N W,Elevated,40.770258,-73.917843,Astoria,Manhattan,1,1,1,NaN,POINT (-73.917843 40.770258)
2,R04,3,3,BMT,Astoria,30 Av,Q,False,N W,Elevated,40.766779,-73.921479,Astoria,Manhattan,0,0,0,NaN,POINT (-73.921479 40.766779)
3,R05,4,4,BMT,Astoria,Broadway,Q,False,N W,Elevated,40.761820,-73.925508,Astoria,Manhattan,0,0,0,NaN,POINT (-73.925508 40.76182)
4,R06,5,5,BMT,Astoria,36 Av,Q,False,N W,Elevated,40.756804,-73.929575,Astoria,Manhattan,0,0,0,NaN,POINT (-73.929575 40.756804)


In [7]:
pois.head()

,the_geom,SEGMENTID,COMPLEXID,SAFTYPE,SOS,PLACEID,FACI_DOM,BIN,BOROUGH,CREATED,MODIFIED,FACILITY_T,SOURCE,B7SC,PRI_ADD,NAME
0,POINT (-74.00701717096757 40.724634757833414),31895,0,N,1.0,567,9,0,1.0,05/14/2009 12:00:00 AM,11/18/2011 12:00:00 AM,6,DoITT,19743001.0,0,HOLLAND
1,POINT (-73.82661642130311 40.797182526598505),306303,3378,N,2.0,568,8,0,4.0,05/14/2009 12:00:00 AM,01/09/2017 12:00:00 AM,6,DoITT,49731001.0,0,WHITESTONE
2,POINT (-73.99395441100663 40.70384707235758),144842,3960,N,2.0,576,8,0,3.0,05/14/2009 12:00:00 AM,01/22/2018 12:00:00 AM,6,DoITT,39734001.0,0,BROOKLYN
3,POINT (-73.9919414213091 40.70960010711745),162664,0,N,1.0,580,8,0,1.0,05/14/2009 12:00:00 AM,05/11/2011 12:00:00 AM,6,DoITT,19795001.0,0,MANHATTAN
4,POINT (-73.9526609766105 40.73906602249743),157362,0,N,1.0,582,8,0,3.0,05/14/2009 12:00:00 AM,03/03/2017 12:00:00 AM,6,DoITT,39740001.0,0,PULASKI


In [8]:
# Convert the geometry column to Shapely Point objects
stations['Georeference'] = stations['Georeference'].apply(loads)

# Extract latitude and longitude from Shapely Point objects
stations['latitude'] = stations['Georeference'].apply(lambda point: point.y)
stations['longitude'] = stations['Georeference'].apply(lambda point: point.x)

# Convert to NumPy array
station_coords = stations[['latitude', 'longitude']].to_numpy()

In [9]:
# Convert the geometry column to Shapely Point objects
pois['the_geom'] = pois['the_geom'].apply(loads)

# Extract latitude and longitude from Shapely Point objects
pois['latitude'] = pois['the_geom'].apply(lambda point: point.y)
pois['longitude'] = pois['the_geom'].apply(lambda point: point.x)

# Convert to NumPy array
poi_coords = pois[['latitude', 'longitude']].to_numpy()

In [10]:
# Build a KDTree for stations
tree = KDTree(station_coords)

In [11]:
# Query the KDTree for each POI to find the closest station
distances, indices = tree.query(poi_coords)

In [12]:
# Add results back to the POI DataFrame
pois['closest_station_id'] = stations.iloc[indices]['Station ID'].values
pois['closest_complex_id'] = stations.iloc[indices]['Complex ID'].values
pois['closest_station_name'] = stations.iloc[indices]['Stop Name'].values
pois['distance_to_station'] = distances  # Approx Cartesian distance

In [13]:
pois.head()

,the_geom,SEGMENTID,COMPLEXID,SAFTYPE,SOS,PLACEID,FACI_DOM,BIN,BOROUGH,CREATED,...,SOURCE,B7SC,PRI_ADD,NAME,latitude,longitude,closest_station_id,closest_complex_id,closest_station_name,distance_to_station
0,POINT (-74.00701717096757 40.724634757833414),31895,0,N,1.0,567,9,0,1.0,05/14/2009 12:00:00 AM,...,DoITT,19743001.0,0,HOLLAND,40.724635,-74.007017,325,325,Canal St,0.001928
1,POINT (-73.82661642130311 40.797182526598505),306303,3378,N,2.0,568,8,0,4.0,05/14/2009 12:00:00 AM,...,DoITT,49731001.0,0,WHITESTONE,40.797183,-73.826616,447,447,Flushing-Main St,0.037737
2,POINT (-73.99395441100663 40.70384707235758),144842,3960,N,2.0,576,8,0,3.0,05/14/2009 12:00:00 AM,...,DoITT,39734001.0,0,BROOKLYN,40.703847,-73.993954,173,173,High St,0.005662
3,POINT (-73.9919414213091 40.70960010711745),162664,0,N,1.0,580,8,0,1.0,05/14/2009 12:00:00 AM,...,DoITT,19795001.0,0,MANHATTAN,40.709600,-73.991941,234,234,East Broadway,0.004479
4,POINT (-73.9526609766105 40.73906602249743),157362,0,N,1.0,582,8,0,3.0,05/14/2009 12:00:00 AM,...,DoITT,39740001.0,0,PULASKI,40.739066,-73.952661,464,464,Vernon Blvd-Jackson Av,0.003677
